In [ ]:
import torch
import torch.nn as nn
class SpectralDecoder(nn.Module):
    def __init__(self, num_P, num_L,kernel_size=1, stride=1, padding=0, bias=False):
        super(SpectralDecoder, self).__init__()
        self.width = num_P
        self.out_channels = num_L

        # 定义一个 3D 卷积层
        self.conv3d = nn.Conv3d(
            in_channels=1,  # 输入通道数
            out_channels=self.out_channels,  # 输出通道数
            kernel_size=(self.width, 1, 1),  # 卷积核大小
            stride=1,  # 步幅
            padding=0  # 无填充
        )

    def forward(self, x):
        x_expanded = x.unsqueeze(1)  # 在维度 1 插入
        output = self.conv3d(x.unsqueeze(1)).squeeze(2)


        return output

    
if __name__ == '__main__':
    # 定义输入和输出维度
    b = 2    # 批次大小
    p = 4    # 输入通道数
    h = 3    # 输入高度
    w = 3    # 输入宽度
    l = 5   # 输出通道数

    # 创建局部连接层实例
    locally_connected = SpectralDecoder(
        num_P=p,
        num_L=l,
        kernel_size=1,    # 根据需求，可以调整 kernel_size
        stride=1,
        padding=0,
        bias=False        # 根据需求，决定是否使用偏置
    )

    # 输入张量 [b, p, h, w]
    x = torch.randn(b, p, h, w)

    # 前向传播
    output = locally_connected(x)

    # 输出形状应为 [b, l, h, w]
    print(output.shape)  # 输出: torch.Size([2,5,3,3])

In [ ]:
class ChannelWise1DEncoder(nn.Module):
    def __init__(self, input_channels, output_channels, kernel_size=3):
        """
        1D 卷积编码器，用于通道间的上下文信息提取
        :param input_channels: 输入通道数
        :param output_channels: 输出通道数
        :param kernel_size: 1D 卷积核大小 
        """
        super(ChannelWise1DEncoder, self).__init__()
        self.conv1d = nn.Conv1d(
            in_channels=input_channels,
            out_channels=output_channels,
            kernel_size=kernel_size,
            stride=1,
            padding=(kernel_size - 1) // 2,  # 确保输入和输出的通道维度长度一致
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        # 将输入的通道维度切换为序列维度 (B, C, H, W) -> (B, H*W, C)
        batch_size, channels, height, width = x.shape
        x = x.permute(0, 2, 3, 1).reshape(batch_size, height * width, channels)  # (B, H*W, C)
        # 应用 1D 卷积
        x = self.conv1d(x)
        x = self.relu(x)
        # 恢复为原始维度 (B, H*W, C_out) -> (B, C_out, H, W)
        x = x.reshape(batch_size, height, width, -1).permute(0, 3, 1, 2)  # (B, C_out, H, W)
        return x

# 测试代码
if __name__ == "__main__":
    input_tensor = torch.randn(4, 128, 25, 25)  # 假设输入为 Lr_HSI (B, C, H, W)
    encoder = ChannelWise1DEncoder(input_channels=128, output_channels=64, kernel_size=3)
    output_tensor = encoder(input_tensor)
    print(f"Output shape: {output_tensor.shape}")  # 应输出: (4, 64, 25, 25)